In [1]:
%pylab inline

import numpy as np
from sklearn.tree import DecisionTreeRegressor
from copy import deepcopy

Populating the interactive namespace from numpy and matplotlib


In [49]:
class LambdaMART:
    def __init__(self, n_estimators, base_estimator=DecisionTreeRegressor(), step_estimator=DecisionTreeRegressor(),
                 alpha=1., beta=1.):
        """
            n_estimators : число деревьев в обучении
            base_estimator : выбор модели для каждой итерации
            step_estimator : выбор модели для предсказания темпа обучения
            alpha : коэффициент регуляризации XGBoost
            beta : коэффициент при предсказании темпа обучения
        """
        self.estimators = []
        self.step_estimators = []
        self.n_estimators = n_estimators
        self.base_estimator = base_estimator
        self.step_estimator = step_estimator
        self.alpha = alpha
        self.beta = beta
        #self.q_dict = defaultdict(lambda : np.zeros(query_d))
        #self.query_d = query_d
        
    def fit(self, X, y, qids, queries, verbose=False):
        """
            X : признаки пар запрос-документ
            y : метки релевантности
            qids : id запросов
            queries : признаки запросов
        """
        self.estimators = []
        self.step_estimators = []
        for i in xrange(self.n_estimators):
            grad = self.loss_grad(self.predict(X), y)
            h = self.double_grad(self.predict(X), y)
            if verbose:
                print "Grad:"
                print grad
            estimator = deepcopy(self.base_estimator)
            estimator.fit(X, -grad/ (self.alpha + h))
            self.estimators.append(estimator)
            if verbose:
                print "Predict:"
                print self.predict(X)
            # Обучим предсказатель шага
            #q_dict = self.generate_query_features(X, qids)
            step_estimator = deepcopy(self.step_estimator)
            # Список всех запросов(уникальных)
            qs = np.unique(qids)
            # Список значений того, что нужно предсказывать
            pred_values = np.zeros(len(qs))
            q_list = np.zeros((len(qs), queries.shape[1]))
            for idx, q in enumerate(qs):
                # Суммируем значения по всем элементам с данным запросом q
                pred_values[idx] = 0.
                for j in xrange(X.shape[0]):
                    if qid == q:
                        q_list[idx] = queries[j]
                        pred_values[idx] += 1. / (1. + self.beta * h[j])            
                #q_list[idx] = q_dict[q]
            step_estimator.fit(q_list, pred_values)
            self.step_estimators.append(step_estimator)
        return self
    
    """def generate_query_features(X, qids):
        q_dict = defaultdict(lambda : np.zeros(query_d))
        unique_q, counts = np.unique(queries, return_counts=True)
        for i,q in enumerate(unique_q):
            for j in xrange(X.shape[0]):
                if queries[j] == q:
                    q_dict[q] += np.append(X[j][0:5], X[j][10:15]) / counts[i]
        return q_dict"""
            
    def predict(self, X, qids, queries, verbose=False):
        y_pred = np.zeros(X.shape[0])
        for est_i in xrange(self.n_estimators):
            est_result = self.n_estimators[est_i].predict(X)
            step = self.step_estimators[est_i].predict(queries)
            for i in xrange(X.shape[0]):
                y_pred[i] += est_result[i] * step
        return y_pred
    
    def loss_grad(self, pred_y, y):
        n_elems = y.shape[0]
        grad = np.zeros(n_elems)
        # id_y - индексы в массиве pred_y, отсортированные по убыванию ранжирующей функции 
        id_y = np.argsort(np.argsort(y)[::-1])
        for i in xrange(n_elems):
            for j in xrange(n_elems):
                buf = 0.
                if y[i] > y[j]:
                    buf = -self.rho(pred_y[i], pred_y[j])
                if y[i] < y[j]:
                    buf = self.rho(pred_y[j], pred_y[i])
                if buf != 0.:
                    grad[i] += np.abs(self.ndcg_replace(y[i], y[j], id_y[i], id_y[j])) * buf
        return grad
        
    def double_grad(self, pred_y, y):
        n_elems = y.shape[0]
        h = np.zeros(n_elems)
        # id_y - индексы в массиве pred_y, отсортированные по убыванию ранжирующей функции 
        id_y = np.argsort(np.argsort(y)[::-1])
        for i in xrange(n_elems):
            for j in xrange(n_elems):
                delta_ndcg = np.abs(self.ndcg_replace(y[i], y[j], id_y[i], id_y[j]))
                h[i] += delta_ndcg * self.rho(pred_y[i], pred_y[j]) * (1 - self.rho(pred_y[i], pred_y[j]))
        return h
                
    def ndcg_replace(self, value_a, value_b, id_a, id_b):
        return (2. ** value_b - 2. ** value_a) * (1./ np.log2(2 + id_a) - 1./ np.log2(2 + id_b))
    
    def rho(self, a, b):
        return 1. / (1. + np.exp(a - b))

In [41]:
np.arange(0., 2., 0.2)

array([ 0. ,  0.2,  0.4,  0.6,  0.8,  1. ,  1.2,  1.4,  1.6,  1.8])

Далее рассмотрим работу на стандартном датасете

In [3]:
# Пример получения индексов элементов в отсортированном в массиве отсортированном по убыванию. Нужно для вычисления метрик
a = [2,3,4,1]

def do_magic(a):
    return np.argsort(np.argsort(a)[::-1])

do_magic(a)

array([2, 1, 0, 3], dtype=int64)

Считаем часть датасета MQ2007

In [7]:
# Проверка на части датасета MQ2007
fin = open('train.txt', 'r')

num_elems = 200
X = np.zeros((num_elems, 46))
y = np.zeros(num_elems)
queries = np.zeros(num_elems)

num = 0
i = 0

lines = fin.readlines()[:num_elems]

for i, line in enumerate(lines):
    parsed = line.split(' ')
    y[i] = int(parsed[0])
    queries[i] = int(parsed[1][4:])
    for j in xrange(46):
        if j < 9:
            X[i][j] = float(parsed[j + 2][2:])
        else:
            X[i][j] = float(parsed[j + 2][3:])
                
# Нормализуем y
y /= np.max(y)

In [43]:
from collections import defaultdict

# Признаки для запросов
query_d = 10 # Число признаков в запросе. Берём 0-4, 10-14 признаки пары и усредняем
q_dict = defaultdict(lambda : np.zeros(query_d)) # dictionary, который по id запроса возвращает его признаки

unique_q, counts = np.unique(queries, return_counts=True)
for i,q in enumerate(unique_q):
    for j in xrange(X.shape[0]):
        if queries[j] == q:
            q_dict[q] += np.append(X[j][0:5], X[j][10:15]) / counts[i]

# Финальная матрица для запросов. По индексу получаем признаки запроса, соответствующие элементу из X с этим индексом
q = np.zeros((num_elems, query_d))
for i in xrange(num_elems):
    q[i] = q_dict[queries[i]]

In [48]:
q

array([[ 0.09802615,  0.13750005,  0.18125   , ...,  0.18866905,
         0.15317262,  0.07571875],
       [ 0.09802615,  0.13750005,  0.18125   , ...,  0.18866905,
         0.15317262,  0.07571875],
       [ 0.09802615,  0.13750005,  0.18125   , ...,  0.18866905,
         0.15317262,  0.07571875],
       ..., 
       [ 0.05055205,  0.025     ,  0.1       , ...,  0.1       ,
         0.025     ,  0.04307292],
       [ 0.05055205,  0.025     ,  0.1       , ...,  0.1       ,
         0.025     ,  0.04307292],
       [ 0.05055205,  0.025     ,  0.1       , ...,  0.1       ,
         0.025     ,  0.04307292]])

In [38]:
# Разобьём выборку на train, validation, test
shuffle_idx = np.random.permutation(num_elems)

In [40]:
X_train = X[shuffle_idx][:100]
X_valid = X[shuffle_idx][100:150]
X_test = X[shuffle_idx][150:200]

y_train = y[shuffle_idx][:100]
y_valid = y[shuffle_idx][100:150]
y_test = y[shuffle_idx][150:200]

q_train = queries[shuffle_idx][:100]
q_valid = queries[shuffle_idx][100:150]
q_test = queries[shuffle_idx][150:200]

In [6]:
def ndcgl(y_pred, y, L=10):
    """
        y_pred : предсказанные значения функции ранжирования
        y : метки релевантности
    """
    dcgl = 0.
    idcgl = 0.
    # Отсортируем значения по убыванию функции ранжирования
    idx = np.argsort(y_pred)[::-1]
    # Метки релевантности для отсортированного массива y_pred
    l = y[idx]
    for i in xrange(L):
        dcgl += (2. ** l[i] - 1) / np.log2(i + 2)
        idcgl += 1 / np.log2(i + 2)
    return dcgl / idcgl

In [62]:
# Небольшой GridSearch.
from numpy import unravel_index

arr_a = [0.1, 0.5, 1., 2.,5.]
arr_L = [1, 2, 5, 7, 10, 20]

results = np.zeros((len(arr_a), len(arr_L)))

for i_a, a in enumerate(arr_a):
    for i_L, L in enumerate(arr_L):
        clf = LambdaMART(L, alpha=a).fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        results[i_a, i_L] = ndcgl(y_pred, y_test)
        print a, L, ndcgl(y_pred, y_test)
        
print "Max at: ", unravel_index(np.argmax(results), results.shape)

0.1 1 0.211887805341
0.1 2 0.22071794876
0.1 5 0.416157258976
0.1 7 0.342168816644
0.1 10 0.342939054669
0.1 20 0.505389034784
0.5 1 0.530775041421
0.5 2 0.497394644788
0.5 5 0.318533164469
0.5 7 0.352536470777
0.5 10 0.434677945375
0.5 20 0.453780324519
1.0 1 0.369109617977
1.0 2 0.4712324459
1.0 5 0.47251167022
1.0 7 0.498465859973
1.0 10 0.462876329244
1.0 20 0.387205454472
2.0 1 0.441542208886
2.0 2 0.45065075968
2.0 5 0.527608142565
2.0 7 0.541846304858
2.0 10 0.491570470471
2.0 20 0.312225088744
5.0 1 0.449811609374
5.0 2 0.451307802016
5.0 5 0.470528237462
5.0 7 0.473860480277
5.0 10 0.4835748785
5.0 20 0.354967777456
Max at:  (3, 3)


In [65]:
arr_a[3], arr_L[3], results[3,3]

(2.0, 7, 0.54184630485795415)

In [34]:
a = np.array([1,5,3,4])
b = np.array([1,0,2,3])
idx = np.argsort(a)[::-1]
a[idx], b[idx]

(array([5, 4, 3, 1]), array([0, 3, 2, 1]))

In [64]:
# Обычный DecisionTreeRegressor
tempclf = DecisionTreeRegressor().fit(X_train, y_train)
ndcgl(tempclf.predict(X_test), y_test)

0.41854413705574878

### LambdaMART из XGBoost

In [3]:
# Необходимые технические строки. Исполняйте их в случае ошибки WindowsError Error 127. 
# Путь заменить на своё расположение mingw-64
dir = r'C:\Program Files\mingw-w64\x86_64-6.2.0-posix-seh-rt_v5-rev1\mingw64\bin'
import os

os.environ['PATH'].find(dir)
os.environ['PATH'] = dir + ';' + os.environ['PATH']

In [4]:
import xgboost as xgb

In [33]:
# Проверка на части датасета MQ2007
fin = open('train.txt', 'r')

num_elems = 1000
X = np.zeros((num_elems, 46))
y = np.zeros(num_elems)

num = 0
i = 0

lines = fin.readlines()[:num_elems]

for i, line in enumerate(lines):
    parsed = line.split(' ')
    # qid
    y[i] = int(parsed[0])
    for j in xrange(46):
        if j < 9:
            X[i][j] = float(parsed[j + 2][2:])
        else:
            X[i][j] = float(parsed[j + 2][3:])

In [34]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [37]:
clf = xgb.XGBClassifier(objective='rank:ndcg')
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

In [43]:
from sklearn.metrics import accuracy_score

accuracy_score(y_pred, y_test)

0.83333333333333337